In [1]:
import os
import json
import glob
import numpy as np
import pandas as pd
import open3d as o3d
import meshplot as mp

In [2]:
base_dir = os.path.dirname(os.getcwd())
data_dir = os.path.join(base_dir, "data")

In [3]:
train_files = glob.glob(os.path.join(data_dir, "original", "train", "*", "*.obj"))
valid_files = glob.glob(os.path.join(data_dir, "original", "val", "*", "*.obj"))
len(train_files), len(valid_files)

(7003, 1088)

In [4]:
def read_objfile(path, return_o3d=False):
    obj = o3d.io.read_triangle_mesh(train_files[0])
    if return_o3d:
        return obj
    else:
        v = np.asarray(obj.vertices, dtype=np.float32)
        f = np.asarray(obj.triangles, dtype=np.int32)
        return v, f

In [5]:
v, f = read_objfile(train_files[0])
v.shape, f.shape

((2487, 3), (1317, 3))

In [6]:
mp.plot(v, f)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [7]:
def redirect_same_vertices(vertices, faces):
    
    faces_with_coord = []
    for face in faces:
        faces_with_coord.append([tuple(vertices[idx]) for idx in face])
    
    coord_to_minimum_vertex = {}
    new_vertices = []
    cnt_new_vertices = 0
    for vertex in vertices:
        vertex_key = tuple(vertex)
        
        if vertex_key not in coord_to_minimum_vertex.keys():
            coord_to_minimum_vertex[vertex_key] = cnt_new_vertices
            new_vertices.append(vertex)
            cnt_new_vertices += 1
    
    new_faces = []
    for face in faces_with_coord:
        new_faces.append([
            coord_to_minimum_vertex[coord] for coord in face
        ])
    
    return np.stack(new_vertices), np.array(new_faces, dtype=np.int32)

In [8]:
v_redirected, f_redirected = redirect_same_vertices(v, f)
v_redirected.shape, f_redirected.shape

((677, 3), (1317, 3))

In [9]:
mp.plot(v_redirected, f_redirected)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [10]:
def reorder_vertices(vertices_df, columns=["z", "y", "x"], add_sorted_index=True):
    # sorting bottom -> top (descending order)
    vertices_df = vertices_df.sort_values(columns, ascending=False)
    
    if add_sorted_index:
        vertices_df["sorted_index"] = np.arange(len(vertices_df))
    return vertices_df

In [11]:
columns = ["z", "y", "x"]
v_df = pd.DataFrame(v_redirected, columns=columns)
v_df_reorder = reorder_vertices(v_df, columns=columns, add_sorted_index=True)
v_df_reorder.shape

(677, 4)

In [12]:
def reorder_faces(vertices_df, faces):
    # this func contains two sorting: in-face-triple sort and all-face sort.
    faces_sorted = []
    for face in faces:
        face_ids = vertices_df.loc[face]["sorted_index"]
        face_ids = np.sort(face_ids)
        faces_sorted.append(face_ids)
    
    # smaller index in face triple means nearer to bottom.
    faces_sorted = pd.DataFrame(faces_sorted)
    faces_sorted = faces_sorted.sort_values(list(range(3)), ascending=True)
    return pd.DataFrame(faces_sorted)

In [13]:
f_reorder = reorder_faces(v_df_reorder, f_redirected).values
v_reorder = v_df_reorder.values[:, :3]
v_reorder.shape, f_reorder.shape

((677, 3), (1317, 3))

In [14]:
mp.plot(v_reorder, f_reorder)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [15]:
def bit_quantization(vertices, bit=8, v_min=-1., v_max=1.):
    # vertices must have values between -1 to 1.
    dynamic_range = 2 ** bit - 1
    discrete_interval = (v_max-v_min) / (dynamic_range)#dynamic_range
    offset = (dynamic_range) / 2
    
    vertices = vertices / discrete_interval + offset
    vertices = np.clip(vertices, 0, dynamic_range-1)
    return vertices.astype(np.int32)

In [16]:
v_quantized = bit_quantization(v_reorder)
v_quantized

array([[160, 136, 130],
       [160, 136, 126],
       [160, 136, 124],
       ...,
       [ 94, 136, 131],
       [ 94, 136, 127],
       [ 94, 136, 124]], dtype=int32)

In [17]:
mp.plot(v_quantized, f_reorder)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(127.0, 12…

In [18]:
def load_pipeline(file_path, bit=8):
    v, f = read_objfile(file_path)
    v, f = redirect_same_vertices(v, f)
    
    columns = ["z", "y", "x"]
    v = pd.DataFrame(v, columns=columns)
    v = reorder_vertices(v, columns=columns, add_sorted_index=True)
    
    f = reorder_faces(v, f)
    v = v.values[:, :3]
    
    v = bit_quantization(v, bit=bit)
    f = f.values
    return v, f

In [19]:
v, f = load_pipeline(train_files[0])

In [20]:
mp.plot(v, f)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(127.0, 12…

In [21]:
classes = ["basket", "chair", "lamp", "sofa", "table"]

In [22]:
for class_ in classes:
    print(class_)
    class_datas = []
    
    for file_path in train_files:
        if file_path.split("/")[-2] == class_:
            v, f = load_pipeline(file_path)
            class_datas.append({
                "vertices": v.tolist(),
                "faces": f.tolist(),
            })
            
    with open(os.path.join(data_dir, "preprocessed", "train", class_+".json"), "w") as fw:
        json.dump(class_datas, fw, indent=4)

basket
chair
lamp
sofa
table


In [23]:
for class_ in classes:
    print(class_)
    class_datas = []
    
    for file_path in valid_files:
        if file_path.split("/")[-2] == class_:
            v, f = load_pipeline(file_path)
            class_datas.append({
                "vertices": v.tolist(),
                "faces": f.tolist(),
            })
            
    with open(os.path.join(data_dir, "preprocessed", "valid", class_+".json"), "w") as fw:
        json.dump(class_datas, fw, indent=4)

basket
chair
lamp
sofa
table
